# PeruCovid19 & LatamCovid19 & DSRP

### 1.- Loading data


### 1.1. Cases Covid-19 per day

In [9]:
!git clone https://github.com/DataScienceResearchPeru/covid-19_latinoamerica.git

Cloning into 'covid-19_latinoamerica'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 4239 (delta 0), reused 5 (delta 0), pack-reused 4234
Receiving objects: 100% (4239/4239), 29.28 MiB | 19.45 MiB/s, done.
Resolving deltas: 100% (2976/2976), done.


In [0]:
# Importamos las librerías
import pandas as pd
import numpy as np
import datetime
import os

import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go

from google.colab import files

path='/content/covid-19_latinoamerica/latam_covid_19_data/daily_reports/'

In [11]:
path, dirs, files = next(os.walk(path))
numero_archivos = len(files) 
print('Hay {} archivos en el folder y 1 el README. Iteraremos {} veces'.format(numero_archivos,numero_archivos-1))

Hay 55 archivos en el folder y 1 el README. Iteraremos 54 veces


In [12]:
# Generamos un arreglo con cada día que se tiene en archivo.
base = (datetime.datetime.today()).date()
numdays = numero_archivos-1
date_list = [str(base - datetime.timedelta(days=x))+str('.csv') for x in range(numdays)]
print('Agregamos {} nombres de archivo al arreglo'.format(len(date_list)))
str(date_list)

Agregamos 54 nombres de archivo al arreglo


"['2020-04-18.csv', '2020-04-17.csv', '2020-04-16.csv', '2020-04-15.csv', '2020-04-14.csv', '2020-04-13.csv', '2020-04-12.csv', '2020-04-11.csv', '2020-04-10.csv', '2020-04-09.csv', '2020-04-08.csv', '2020-04-07.csv', '2020-04-06.csv', '2020-04-05.csv', '2020-04-04.csv', '2020-04-03.csv', '2020-04-02.csv', '2020-04-01.csv', '2020-03-31.csv', '2020-03-30.csv', '2020-03-29.csv', '2020-03-28.csv', '2020-03-27.csv', '2020-03-26.csv', '2020-03-25.csv', '2020-03-24.csv', '2020-03-23.csv', '2020-03-22.csv', '2020-03-21.csv', '2020-03-20.csv', '2020-03-19.csv', '2020-03-18.csv', '2020-03-17.csv', '2020-03-16.csv', '2020-03-15.csv', '2020-03-14.csv', '2020-03-13.csv', '2020-03-12.csv', '2020-03-11.csv', '2020-03-10.csv', '2020-03-09.csv', '2020-03-08.csv', '2020-03-07.csv', '2020-03-06.csv', '2020-03-05.csv', '2020-03-04.csv', '2020-03-03.csv', '2020-03-02.csv', '2020-03-01.csv', '2020-02-29.csv', '2020-02-28.csv', '2020-02-27.csv', '2020-02-26.csv', '2020-02-25.csv']"

In [13]:
# Convertimos a data frame cada archivo.
array_dataframes=[]
indice=0
print('En total hay {} archivos'.format(numdays))
for date in date_list:
  array_dataframes.append(pd.read_csv(path+date, sep=','))    
  array_dataframes[indice]['Date']=date[:-4]
  del array_dataframes[indice]['Last Update']
  del array_dataframes[indice]['Country']
  del array_dataframes[indice]['Subdivision']
  # Crear dataframe total
  indice+=1

En total hay 54 archivos


In [0]:
total_data=pd.DataFrame() # structure 
for m in range(len(array_dataframes)-1):
  total_data=pd.concat([total_data,array_dataframes[m]], ignore_index=True)  
# Eliminamos duplicados
total_data=total_data.drop_duplicates(keep=False)

Loading ISO 3166-2 Code

In [0]:
url='https://raw.githubusercontent.com/DataScienceResearchPeru/covid-19_latinoamerica/master/utils/iso3166-2.csv'
template_isocode=pd.read_csv(url, sep=',')
isocode=template_isocode['Code']
array_isocode=isocode.to_numpy()

Checking errors

In [0]:
# Error in confirmed cases
errors_confirmed=pd.DataFrame()
errors_confirmed['Subdivision']=None
errors_confirmed['Date']=None
errors_confirmed['From']=None
errors_confirmed['To']=None

# Errors in deaths cases
errors_deaths=pd.DataFrame()
errors_deaths['Subdivision']=None
errors_deaths['Date']=None
errors_deaths['From']=None
errors_deaths['To']=None

# Errors in recovered cases
errors_recovered=pd.DataFrame()
errors_recovered['Subdivision']=None
errors_recovered['Date']=None
errors_recovered['From']=None
errors_recovered['To']=None

for code in array_isocode:
  # NaN to zero
  per_subdivision=total_data[total_data['ISO 3166-2 Code']==code]
  per_subdivision=per_subdivision.fillna(0)
  per_subdivision=per_subdivision.sort_values(by=['Date'])
  # Variables iniciales
  init_confirmed,init_deaths,init_recovered=0,0,0
  for index, row in per_subdivision.iterrows():
    temp_c=int(row['Confirmed'])
    temp_d=int(row['Deaths'])
    temp_r=int(row['Recovered'])
    day=row['Date']
    # Compare day after vs before in confirmed
    if temp_c>=init_confirmed:
      init_confirmed=temp_c
    else:
      errors_confirmed.loc[index]=[code,day,init_confirmed,temp_c]
    # Compare day after vs before in deaths
    if temp_d>init_deaths:
      init_deaths=temp_d
    else:
      errors_deaths.loc[index]=[code,day,init_deaths,temp_d]
    # Compare day after vs before in recovered
    if temp_r>init_recovered:
      init_recovered=temp_r
    else:
      errors_recovered.loc[index]=[code,day,init_recovered,temp_r]

# Generate a report
errors_confirmed.reset_index(drop=False)   
errors_deaths.reset_index(drop=False)
errors_recovered.reset_index(drop=False)

today=datetime.date.today()

errors_confirmed.to_csv('Error confirmed generated {}.csv'.format(today), sep=',',index=True)
errors_deaths.to_csv('Error deaths generated {}.csv'.format(today), sep=',',index=True)
errors_recovered.to_csv('Error recovered generated {}.csv'.format(today), sep=',',index=True)

files.download('Error confirmed generated {}.csv'.format(today))
files.download('Error deaths generated {}.csv'.format(today))
files.download('Error recovered generated {}.csv'.format(today))

In [0]:
today=datetime.date.today()

errors_confirmed.to_csv('Error confirmed generated {}.csv'.format(today), sep=',',index=True)
errors_deaths.to_csv('Error deaths generated {}.csv'.format(today), sep=',',index=True)
errors_recovered.to_csv('Error recovered generated {}.csv'.format(today), sep=',',index=True)

files.download('Error confirmed generated {}.csv'.format(today))
files.download('Error deaths generated {}.csv'.format(today))
files.download('Error recovered generated {}.csv'.format(today))

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Example of plotting errors

In [27]:

iso_name=array_isocode[15]

total_data.sort_values(by=['ISO 3166-2 Code']).reset_index(drop=True)

bar_data = total_data.groupby(['ISO 3166-2 Code','Date'])['Confirmed','Deaths','Recovered'].sum().reset_index().sort_values(by=['Date','ISO 3166-2 Code'], ascending=True)

fig = px.bar(bar_data[bar_data['ISO 3166-2 Code']==iso_name], x="Date", y="Confirmed",color='ISO 3166-2 Code', text = 'Confirmed', orientation='v', height=600,
             title='Cases')
fig.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

